# Accelerating C/C++ code with CUDA on GPUs

In this self-paced, hands-on lab, we will use CUDA C/C++ to accelerate code on NVIDIA GPUs.

Created by Mark Ebersole @ NVIDIA and Edoardo Baldi.


### Notes on usage

- For every step in this guided notebook, there's a corresponding solution in the right folder. A suggested practice would be to copy & rename the original source file, modify it, and the compile it following the hints here. You can always double-check with the solution provided.

- To compile & run all the examples in this notebook, you need a working installation of the CUDA Toolkit, that comes with the `nvcc` compiler (and all the rest, including a ton of very useful examples written by NVIDIA developers). If you don't own it, you can find how to download and install it [HERE](https://developer.nvidia.com/cuda-downloads). 

- **IMPORTANT**: a NVIDIA graphics driver must be installed to be able to **run** properly the examples, if you are not running these on a cluster. In the latter case, everything should be already set up (you may have to load some modules, like `cuda` on Deneb).

- **IMPORTANT 2**: to compile & run all the examples, you can either evaluate the cells here in the notebook or, in a terminal, go into the corresponding directory and issue `make`.

We're going to use one Deneb's GPU node. Each is equipped with a *NVIDIA Tesla K40m (Kepler architecture)*.

In [1]:
!nvidia-smi

Wed Aug 30 10:24:02 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 375.26                 Driver Version: 375.26                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GT 730      Off  | 0000:01:00.0     N/A |                  N/A |
| 35%   39C    P0    N/A /  N/A |    270MiB /   971MiB |     N/A      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Cleanup previous compiled codes

Just go into the root directory (the `sources` directory) and issue `make clean`.

## Writing and Launching GPU Kernels

### Task #1

We're going to be accelerating the ever popular SAXPY (**S**ingle-precision **A** times **X** **P**lus **Y**) function on the GPU using CUDA C/C++.

Using the concepts introduced in Task #1, modify the following code to run on the GPU.  The `TODO` text in the code will help you focus on the appropriate areas that need modification.  You'll probably notice two new API calls in the code; `cudaMallocManaged` and `cudaFree`.  These two functions are working with managed memory using CUDA's Unified Memory system.  We'll explore this in the last task of this lab.  For the moment, you just need to know that they are replacing `malloc` and `free`, respectively.

Open the **task2.cu** file and begin working.  If you get stuck, or just want to check your answer, feel free to look at the **task2_solution.cu** file.

In [12]:
# Execute this cell to compile & run task2.cu
!nvcc -arch=sm_35 -o task2_out.x sources/1-writing_kernel/task2.cu -run

c[0] = 5, c[1] = 5, c[2] = 5, c[3] = 5, c[4] = 5, 
c[4194299] = 5, c[4194300] = 5, c[4194301] = 5, c[4194302] = 5, c[4194303] = 5, 


The output of your program should be all 5's.

### Task #2

The next function we will accelerate is a basic matrix multiplication function.  In this simplified example, we'll assume our matrices are all square - they have the same number of rows and columns.  Your goal is to modify the `matrixMulGPU` function with CUDA so it will run on the GPU.  However, there is a new twist!  Instead of just using one-dimensional blocks of threads and blocks, we'll be using two dimensions; x and y.  So, in addition to using `blockIdx.x`, `blockDim.x`, and `threadIdx.x`, you'll also need to use <code>blockIdx.<span style="color:orange">**y**</span></code>, <code>blockDim.<span style="color:orange">**y**</span></code>, and <code>threadIdx.<span style="color:orange">**y**</span></code>. 

Besides replacing the `TODO` blocks in the `matrixMulGPU` function, you will need to finish initializing the `number_of_blocks` variable in the `main` function to launch the appropriate number of thread blocks.  This variable has a `dim3` type; see the [CUDA documentation](http://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#dim3) for more info on this type.  This is all marked with `TODO` in the code.

While working on **task3.cu** you can always check the **task3_solution.cu file** to see the final answer or if you get stuck.

**Note**: do not modify the CPU version `matrixMulCPU`.  This is used to verify the results of the GPU version.

In [19]:
# Execute this cell to compile & run task3.cu
!nvcc -arch=sm_35 -o task3_out.x sources/2-2d_kernel/task3.cu -run

Success!


## Error Handling

If you started changing the number of blocks and threads per block in the above examples, you may have noticed some cases where you were not getting the expected answer.  Up to this point we have not added any type of error checking, which makes it very hard to tell why a problem is occurring.  Error checking is just as important when programming for a GPU as for a CPU.  So let's add some error checking to the SAXPY example and see if we can introduce some errors to catch.

**Note**: In order to focus on a specific topic, and to keep code and instructions short, a number of examples in this lab leave out error checking code.  However, it is highly encouraged that you include error checking in your code whenever possible!

### Task #3

The SAXPY code for this task is slightly different than in Task #2.  For starters, we are storing the result of our computation back into the `y` array instead of a separate array.  We're also using `float` values for our arrays instead of integers.  On the host side, after calculating our SAXPY result, the code will check for correctness.  If everything is working, the program will print `Max Error: 0.00000`.

Now let's discuss the different types of errors we need to check for.  Kernels are launched asynchronously with respect to the host, meaning that control returns to the host code immediately **after** a kernel is launched but **before** the kernel completes. As a result we need to check for two types of errors; synchronous errors dealing with the kernel launch and asynchronous errors that can happen during kernel execution. To check for synchronous errors we use the function `cudaGetLastError()`, which returns an integer error code. In the example below we compare the result of `cudaGetLastError()` and if it does not equal `cudaSuccess` (defined in the `cudaFor` module), we have an error:

```cpp
cudaError_t ierrSync;
...
myKernel<<<grid, blocks>>>(...);
ierrSync = cudaGetLastError();
if (ierrSync != cudaSuccess) { printf("Sync error: %s\n", cudaGetErrorString(ierrSync)); }
```

To check for an error during execution we need to wait for the kernel to complete.  This is accomplished using the function `cudaDeviceSynchronize()`. As with all CUDA API calls, `cudaDeviceSynchronize()` returns an integer error code and used in this fashion will capture any error that occurs on the device while executing the kernel.  Example usage:

```cpp
cudaError_t ierrAsync;
...
myKernel<<<grid, blocks>>>(...);
ierrAsync = cudaDeviceSynchronize();
if (ierrAsync != cudaSuccess) { printf("Async error: %s\n", cudaGetErrorString(ierrSync)); }
```
    
In addition to the above code changes, there is a logic bug introduced in the code below.  Your objective for Task #4 is to add both types of error checking to the **task4.cu** source code and try to find the bug.  Before modifying any code, compile and run the program as it is and see the result.  Then add error checking and see if you can figure out what the bug is and fix it.  As always, the solution (**task4_solution.cu**) is provided so you can check your work, or look at if you get stuck.

In [24]:
# Execute this cell to compile & run task4.cu
!nvcc -arch=sm_35 -o task4_out.x sources/3-errors/task4.cu -run

Max Error: 0.00000

It's very important to consistently add error checking into your code.  We routinely get questions regarding problems with someone's code, only to see there is no error checking.  Almost always once error checking is added, the problem becomes obvious and is easily fixed.  

## Querying the GPU Device

The CUDA C/C++ device management API allows a programmer to query the number of devices available on a system and the features and capabilities of each device. The simple code below illustrates use of the device management API. After the number of CUDA-enabled devices attached to the system is determined via `cudaGetDeviceCount()`, a loop over these devices is performed (note that devices are enumerated from 0) and the function `cudaGetDeviceProperties()` is used to return information about a device in a variable of type `cudaDeviceProp`. 

```cpp
function deviceQuery
{
  cudaDeviceProp prop;
  int nDevices=0, i; 
  cudaError_t ierr;

  ierr = cudaGetDeviceCount(&nDevices);
 
  for( i = 0, i < nDevices; ++i )
  {
     ierr = cudaGetDeviceProperties(&prop, i);
     printf("Device number: %d\n", i);
     printf("  Device name: %s\n", prop.name);
     printf("  Compute capability: %d.%d\n", prop.major, prop.minor);
     printf("  Max threads per block: %d\n", prop.maxThreadsPerBlock);
     printf("  Max threads in X-dimension of block: %d\n", prop.maxThreadsDim[0]);
     printf("  Max threads in Y-dimension of block: %d\n", prop.maxThreadsDim[1]);
     printf("  Max threads in Z-dimension of block: %d\n\n", prop.maxThreadsDim[2]);
  }
}
```

The type `cudaDeviceProp` contains many fields.  Only a few are demonstrated in this code. One value demonstrated is the compute capability, which indicates the GPU architecture and is given in `major.minor` format. The major component reflects the generation of the architecture, and the minor component the revision within that generation. All Fermi generation GPUs have a compute capability of 2.x,  Kepler GPUs 3.x, and Maxwell GPUs 5.x. In addition to the compute capability, this code prints the maximum number of threads per block possible as well as the maximum number of threads in each dimension of the block.

### Task #4

For our next task, let's modify a 2D version of the SAXPY code to check for valid launch configuration arguments before launching the kernel.  The launch configuration parameters to check are the number of threads per block and number of blocks, in both the X and Y dimension.  See the solution if needed.

In [1]:
# Execute this cell to compile & run task5.cu
!nvcc -arch=sm_35 -o task5_out.x sources/4-device_properties/task5.cu -run

========== DEVICE PROPERTIES ==========
Device number: 0
  Device name: GeForce GT 730
  Compute capability: 3.5
  Max threads per block: 1024
  Max threads in X-dimension of block: 1024
  Max threads in Y-dimension of block: 1024
  Max threads in Z-dimension of block: 64

Max number of threads exceeded!


Try entering different sizes in the block dimension line, `dim3 threads_per_block(32,16,1);`, and make sure your new GPU device property check works correctly!

As you begin writing GPU code that could possibly run on multiple or different types of GPUs, you should use the ability to easily query each device to determine the optimal configuration for your code.

## Memory Management

It is important to realize that the GPU has it's own physical memory; just like the CPU uses system RAM for it's memory.  When executing code on the GPU, we have to ensure any data it needs is first copied across the PCI-Express bus to the GPU's memory before we launch our kernels.

Before the release of CUDA 6, it was a requirement that you, the developer, handle allocation and movement of data between the two memory spaces.  This was, and can still be done with the following minimal set of API calls (detailed documentation [here](http://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY) at [docs.nvidia.com](http://docs.nvidia.com/)):

* `cudaMalloc ( void** devPtr, size_t size )` - this API call is used to allocate memory on the GPU, and is very similar to using `malloc` on the CPU.  You provide the address of a pointer that will point to the memory after the call completes successfully, as well as the number of bytes to allocate.
* `cudaMemcpy ( void* dst, const void* src, size_t count, cudaMemcpyKind kind )` - also very similar the standard `memcpy`, this API call is used to copy data between the CPU and GPU.  It takes a destination pointer, a source pointer, the number of bytes to copy, and the fourth parameter indicates which direction the data is traveling: GPU->CPU, CPU->GPU, or GPU->GPU.
* `cudaFree ( void* devPtr )` - we use this API call to free any memory we allocated on the GPU.

Starting in CUDA 6, this required manual handling of data movement has been removed with the release of Unified Memory.  Unified Memory creates an area of managed memory, and the underlying system handles moving this managed data between CPU and GPU memory spaces when required.  Because of this, getting an application executing on an NVIDIA GPU with CUDA C/C++ has become a much quicker and efficient process - including simpler to maintain code.

To make use of this managed memory area with Unified Memory, you use the following API calls.

* `cudaMallocManaged ( T** devPtr, size_t size );` - allocate `size` bytes in managed memory and store in devPtr.
* `cudaFree ( void* devPtr )` - we use this API call to free any memory we allocated in managed memory.

Once you have used `cudaMallocManaged` to allocate some data, you just use the pointer in your code, regardless of whether it's the CPU or GPU accessing the data.  Before Unified Memory, you typically had two pointers associated with data; one for CPU memory and one for GPU memory (usually using the GPU name preceded with a `d_` to indicate device memory).

Managed memory is synchronized between memory spaces at kernel launch and any device synchronization points.  This means that on Kepler and Maxwell architectures an explicit synchronization point (typically `cudaDeviceSynchronize()`) needs to be inserted after a kernel launch but before the host will use data that has been generated by that kernel.

### Task #5

For this task, you need to modify the code in task6.cu so it makes use of Unified Memory instead of the manual data management it currently uses.  If you get stuck, there you can always look to the solution (task6_solution.cu).
<pre>On device: name=hello, value=10
On host: name=dello, value=11
</pre>

In [34]:
# Execute this cell to compile & run task6.cu
!nvcc -arch=sm_35 -o task6_out.x sources/5-memory_management/task6.cu -run

On device: name=hello, value=10
On host: name=dello, value=11


You can see why Unified Memory is so appealing - it removes the requirement for complex data management code.  Allowing you to get your functions executing on the GPU with less developer effort.

## Learn More

If you are interested in learning more, you can use the following resources:

* Learn more at the [CUDA Developer Zone](https://developer.nvidia.com/category/zone/cuda-zone).
* If you have an NVIDIA GPU in your system, you can download and install the [CUDA tookit](https://developer.nvidia.com/cuda-toolkit).
* Take the fantastic online and **free** Udacity [Intro to Parallel Programming](https://www.udacity.com/course/cs344) course which uses CUDA C.
* Search or ask questions on [Stackoverflow](http://stackoverflow.com/questions/tagged/cuda) using the cuda tag